In [1]:
# Desired sample rate
sample_rate=25
resample_rate=4

# Moving average
MA=10

# tsfresh window
max_window=10*resample_rate

In [2]:
import pandas as pd
import numpy as np
import ruptures as rpt
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.legend_handler as handler

from pandas.plotting import autocorrelation_plot

from tsfresh import extract_features, select_features, feature_extraction,extract_relevant_features
from tsfresh.utilities.dataframe_functions import roll_time_series, make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute

from pywt import wavedec, waverec

import itertools
import os
import re
from os import listdir

In [3]:
def filter_haar(data):
    #construct wavelets
    #wavelet='db4' 
    wavelet='haar'
    coeffs = wavedec(data.values, wavelet)
 
    # threshold
    threshold = 1 #( in percentage)

    # perform the thresholding
    coeffs_thresholded = [np.maximum(np.abs(c) - threshold, 0) * np.sign(c) for c in coeffs]
 
    # reconstruct time domain
    time_series_filtered = waverec(coeffs_thresholded, wavelet)
   
    return time_series_filtered


# Make an array with x,y,z acc and gyro and alpha_r and beta_r to use in ruptures
    
def get_signals(df,ts_start,ts_end,filter):
 
    if filter=='haar':
        acc_x_r_c=np.array(filter_haar(df['acc_x_r'][ts_start:ts_end]))
        acc_y_r_c=np.array(filter_haar(df['acc_y_r'][ts_start:ts_end]))
        acc_z_r_c=np.array(filter_haar(df['acc_z_r'][ts_start:ts_end]))

        acc_x_n_c=np.array(filter_haar(df['acc_x_n'][ts_start:ts_end]))
        acc_y_n_c=np.array(filter_haar(df['acc_y_n'][ts_start:ts_end])) 
        acc_z_n_c=np.array(filter_haar(df['acc_z_n'][ts_start:ts_end]))

        gyr_x_n_c=np.array(filter_haar(df['gyr_x_n'][ts_start:ts_end]))
        gyr_y_n_c=np.array(filter_haar(df['gyr_y_n'][ts_start:ts_end]))
        gyr_z_n_c=np.array(filter_haar(df['gyr_z_n'][ts_start:ts_end]))

        gyr_x_r_c=np.array(filter_haar(df['gyr_x_r'][ts_start:ts_end]))
        gyr_y_r_c=np.array(filter_haar(df['gyr_y_r'][ts_start:ts_end]))
        gyr_z_r_c=np.array(filter_haar(df['gyr_z_r'][ts_start:ts_end]))

        alpha_r_c = np.array(filter_haar(df['alpha_r'][ts_start:ts_end]))
        beta_r_c  = np.array(filter_haar(df['beta_r'][ts_start:ts_end]))
    
    else:
        acc_x_r_c=np.array(df['acc_x_r'][ts_start:ts_end])
        acc_y_r_c=np.array(df['acc_y_r'][ts_start:ts_end])
        acc_z_r_c=np.array(df['acc_z_r'][ts_start:ts_end])

        acc_x_n_c=np.array(df['acc_x_n'][ts_start:ts_end])
        acc_y_n_c=np.array(df['acc_y_n'][ts_start:ts_end])
        acc_z_n_c=np.array(df['acc_z_n'][ts_start:ts_end])

        gyr_x_n_c=np.array(df['gyr_x_n'][ts_start:ts_end])
        gyr_y_n_c=np.array(df['gyr_y_n'][ts_start:ts_end])
        gyr_z_n_c=np.array(df['gyr_z_n'][ts_start:ts_end])

        gyr_x_r_c=np.array(df['gyr_x_r'][ts_start:ts_end])
        gyr_y_r_c=np.array(df['gyr_y_r'][ts_start:ts_end])
        gyr_z_r_c=np.array(df['gyr_z_r'][ts_start:ts_end])

        alpha_r_c = np.array(df['alpha_r'][ts_start:ts_end])
        beta_r_c  = np.array(df['beta_r'][ts_start:ts_end])

    # Smoothing with Moving average of (MA)
    window = np.ones(MA*resample_rate) / (MA*resample_rate)
    acc_x_r_c = np.convolve(acc_x_r_c, window, mode='valid')
    acc_y_r_c = np.convolve(acc_y_r_c, window, mode='valid')
    acc_z_r_c = np.convolve(acc_z_r_c, window, mode='valid')

    acc_x_n_c = np.convolve(acc_x_n_c, window, mode='valid')
    acc_y_n_c = np.convolve(acc_y_n_c, window, mode='valid')
    acc_z_n_c = np.convolve(acc_z_n_c, window, mode='valid')

    gyr_x_r_c = np.convolve(gyr_x_r_c, window, mode='valid')
    gyr_y_r_c = np.convolve(gyr_y_r_c, window, mode='valid')
    gyr_z_r_c = np.convolve(gyr_z_r_c, window, mode='valid')

    gyr_x_n_c = np.convolve(gyr_x_n_c, window, mode='valid')
    gyr_y_n_c = np.convolve(gyr_y_n_c, window, mode='valid')
    gyr_z_n_c = np.convolve(gyr_z_n_c, window, mode='valid')

    alpha_r_c = np.convolve(alpha_r_c, window, mode='valid')
    beta_r_c = np.convolve(beta_r_c, window, mode='valid') 
    alpha_min_beta_c=alpha_r_c-beta_r_c

    # Normalise
    alpha_r = (alpha_r_c - alpha_r_c.min()) * (2/(alpha_r_c.max() - alpha_r_c.min())) - 1
    beta_r = (beta_r_c - beta_r_c.min()) * (2/(beta_r_c.max() - beta_r_c.min())) - 1
    alpha_min_beta = (alpha_min_beta_c - alpha_min_beta_c.min()) * (2/(alpha_min_beta_c.max() - alpha_min_beta_c.min())) - 1

    acc_x_n = (acc_x_n_c - acc_x_n_c.min()) * (2/(acc_x_n_c.max() - acc_x_n_c.min())) - 1 
    acc_y_n = (acc_y_n_c - acc_y_n_c.min()) * (2/(acc_y_n_c.max() - acc_y_n_c.min())) - 1
    acc_z_n = (acc_z_n_c - acc_z_n_c.min()) * (2/(acc_z_n_c.max() - acc_z_n_c.min())) - 1

    acc_x_r = (acc_x_r_c - acc_x_r_c.min()) * (2/(acc_x_r_c.max() - acc_x_r_c.min())) - 1 
    acc_y_r = (acc_y_r_c - acc_y_r_c.min()) * (2/(acc_y_r_c.max() - acc_y_r_c.min())) - 1
    acc_z_r = (acc_z_r_c - acc_z_r_c.min()) * (2/(acc_z_r_c.max() - acc_z_r_c.min())) - 1

    gyr_x_n = (gyr_x_n_c - gyr_x_n_c.min()) * (2/(gyr_x_n_c.max() - gyr_x_n_c.min())) - 1 
    gyr_y_n = (gyr_y_n_c - gyr_y_n_c.min()) * (2/(gyr_y_n_c.max() - gyr_y_n_c.min())) - 1
    gyr_z_n = (gyr_z_n_c - gyr_z_n_c.min()) * (2/(gyr_z_n_c.max() - gyr_z_n_c.min())) - 1

    gyr_x_r = (gyr_x_r_c - gyr_x_r_c.min()) * (2/(gyr_x_r_c.max() - gyr_x_r_c.min())) - 1 
    gyr_y_r = (gyr_y_r_c - gyr_y_r_c.min()) * (2/(gyr_y_r_c.max() - gyr_y_r_c.min())) - 1
    gyr_z_r = (gyr_z_r_c - gyr_z_r_c.min()) * (2/(gyr_z_r_c.max() - gyr_z_r_c.min())) - 1
  
 
    return acc_x_n,acc_y_n,acc_z_n,gyr_x_n,gyr_y_n,gyr_z_n,acc_x_r,acc_y_r,acc_z_r,gyr_x_r,gyr_y_r,gyr_z_r,alpha_r,beta_r,alpha_min_beta

# Function to produce the changepoints with ruptures library (pelt or window method)
def segment(stacked_data,window_size,penalty,method):

    #Window-based approach 
    if method=='window':
        algo = rpt.Window(width=window_size, model='rbf').fit(stacked_data)
        results = algo.predict(pen=penalty) 

    # Radial basis function kernel
    if method=='pelt':
        algo=rpt.Pelt(model='rbf').fit(stacked_data)
        results=algo.predict(pen=penalty) 
    
    return results

# Function to replace 2 nearby changepoints with one in the middle
def clean_results(all_results,method,datapoints):
    new_results = []
    i = 0
    while i < len(all_results) - 1:
        if all_results[i+1] - all_results[i] <= datapoints*resample_rate:
            if method=='avg':
                method_result = (all_results[i] + all_results[i+1])/2
            if method=='first':
                method_result=all_results[i]
            if method=='last':
                method_result=all_results[i+1]
            new_results.append(method_result)
            i += 2
        else:
            new_results.append(all_results[i])
            i += 1

    # Add the last value to the new list if it wasn't already added
    if i == len(all_results) - 1:
        new_results.append(all_results[-1])
    new_results = [*set(new_results)]
    new_results.sort()   
    return new_results

# Function to get the changepoint, fine-tuned from analysing labeled data
def get_changepoints():
    
    all_results=[]

    method='window'

    
    
    # Add acc_x_n and y from neck 
    penalty=2.1*resample_rate
    window_size=32*resample_rate
    stacked_data=np.vstack([alpha_r,beta_r,alpha_min_beta]).T 

    results=segment(stacked_data,window_size,penalty,method)
    all_results.append(results)

    # Add acc_x_n and y from neck 
    penalty=1.2*resample_rate
    window_size=32*resample_rate
    stacked_data=np.vstack([acc_x_n,acc_y_n]).T 

    results=segment(stacked_data,window_size,penalty,method)
    all_results.append(results)

    # Now add  acc_z_n  from neck 
    penalty=.87*resample_rate
    window_size=32*resample_rate
    stacked_data=np.vstack([acc_z_n]).T

    results=segment(stacked_data,window_size,penalty,method)
    all_results.append(results)

    # Now add acc from belt  
    penalty=.8*resample_rate
    window_size=32*resample_rate
    stacked_data=np.vstack([acc_x_r,acc_y_r,acc_z_r]).T

    results=segment(stacked_data,window_size,penalty,method)
    all_results.append(results)

    # Now add gyr_xxx_r
    penalty= 2.1*resample_rate
    window_size=32*resample_rate
    stacked_data=np.vstack([gyr_x_r,gyr_y_r,gyr_z_r]).T

    results=segment(stacked_data,window_size,penalty,method)
    all_results.append(results)

    # Now add  gyr_xxx_n  
    penalty= 2.1*resample_rate
    window_size=32*resample_rate
    stacked_data=np.vstack([gyr_x_n,gyr_y_n,gyr_z_n]).T

    results=segment(stacked_data,window_size,penalty,method)
    all_results.append(results)
 
    all_results = [item for sublist in all_results for item in sublist]
    all_results = [*set(all_results)]

    all_results.sort()   

    all_results=clean_results(all_results,'avg',5*resample_rate)
    all_results=clean_results(all_results,'avg',6*resample_rate) 

    return all_results
    



In [4]:
# PLOT FUNCTIONS

# Plot the graph with changepoints for the full dataset
def plot_changepoints(signal1,signal2,signal3,signal4,signal5,signal6,results,range_start,range_end,filename):

    # Draw the gyroscope plot
    fig,ax=plt.subplots(figsize=(16, 4))
    plt.title(f'Changepoints (window method)')

    axb=ax.twinx()

    signal_list=[signal1,signal2,signal3,signal4,signal5,signal6]
    ax.plot(signal_list[0],color='tab:red',linewidth=.25 , label ='x' , alpha=1)
    ax.plot(signal_list[1],color='tab:orange',linewidth=.25 , label ='y' , alpha=1)
    ax.plot(signal_list[2],color='tab:cyan',linewidth=.25 , label ='z'  , alpha=1)
    ax.set_xlim(range_start,range_end)
    axb.plot(signal_list[3],color='tab:green',linewidth=1.6 , label ='alpha')
    axb.plot(signal_list[4],color='tab:blue',linewidth=1.6 , label ='beta')
    axb.plot(signal_list[5],color='tab:red',linewidth=1.6 , label ='a_min_b')
    ax.legend(loc='upper left', bbox_to_anchor=(0, 1))
    axb.legend(loc='upper right', bbox_to_anchor=(1, 1))

    # Draw the change points
    cnt=int(0)
    for r in results:
        ax.axvline(x=r,color="tab:red",linewidth=3,linestyle=':')
        cnt+=1

    fig.savefig(f'../data/processed/output/{filename}.png')
 
# Function to get signals in a range, for use in the frames
def get_signals_range(variables, start, end):
    variables_range = {}
    for key, value in variables.items():
        variables_range[key] = value[start:end]
    return variables_range 

# Function to extract a frame
def get_frame(start,end):

    acc_x_n=np.array(df['acc_x_n'][start:end])
    acc_y_n=np.array(df['acc_y_n'][start:end])
    acc_z_n=np.array(df['acc_z_n'][start:end])
    gyr_x_n=np.array(df['gyr_x_n'][start:end])
    gyr_y_n=np.array(df['gyr_y_n'][start:end])
    gyr_z_n=np.array(df['gyr_z_n'][start:end])

    alpha_r=np.array(df['alpha_r'][start:end])
    beta_r = np.array(df['beta_r'][start:end])
     
    # Normalise
    alpha_r = (alpha_r - alpha_r_c.min()) * (2/(alpha_r_c.max() - alpha_r_c.min())) - 1
    beta_r = (beta_r - beta_r_c.min()) * (2/(beta_r_c.max() - beta_r_c.min())) - 1

    gyr_x_n = (gyr_x_n - gyr_x_n_c.min()) * (2/(gyr_x_n_c.max() - gyr_x_n_c.min())) 
    gyr_y_n = (gyr_y_n - gyr_y_n_c.min()) * (2/(gyr_y_n_c.max() - gyr_y_n_c.min())) 
    gyr_z_n = (gyr_z_n - gyr_z_n_c.min()) * (2/(gyr_z_n_c.max() - gyr_z_n_c.min())) 

    return acc_x_n,acc_y_n,acc_z_n,gyr_x_n,gyr_y_n,gyr_z_n,alpha_r,beta_r

# Function to plot a graph of a frame
def plot_frame(acc_x_n,acc_y_n,acc_z_n,gyr_x_n,gyr_y_n,gyr_z_n,alpha_r,beta_r,i):
    fig,ax=plt.subplots(figsize=(16, 4))

    plt.title(f'frame {i+1}')

    axb=ax.twinx()
    ax.plot(acc_x_n,color='tab:red',linewidth=.9 , label ='acc (x)', alpha=0.4)
    ax.plot(acc_y_n,color='tab:orange',linewidth=.9 , label ='acc (y)', alpha=0.4)
    ax.plot(acc_z_n,color='tab:cyan',linewidth=.9 , label ='acc (z)' , alpha=0.4)

    ax.plot(gyr_x_n,color='tab:red',linewidth=.9 , label ='gyr (x)', linestyle=':', alpha=0.66)
    ax.plot(gyr_y_n,color='tab:orange',linewidth=.9 , label ='gyr (y)', linestyle=':', alpha=0.66)
    ax.plot(gyr_z_n,color='tab:cyan',linewidth=.9 , label ='gyr (z)' , linestyle=':', alpha=0.66)

    axb.plot(alpha_r,color='tab:green',linewidth=1.6 , label ='alpha')
    axb.plot(beta_r,color='tab:blue',linewidth=1.6 , label ='beta')
    ax.legend(loc='upper left', bbox_to_anchor=(0, 1))
    axb.legend(loc='upper right', bbox_to_anchor=(1, 1))
    
    plt.show()

In [5]:
# Function that returns the extracted features for all frames in a file as 1 dimensional elements
 
def get_extracted_features_table(df):
 
    # Create the columns with the features for each signal
    signal_list=['acc_x_n','acc_y_n','acc_z_n','acc_x_r','acc_y_r','acc_z_r','gyr_x_n','gyr_y_n','gyr_z_n','gyr_x_r','gyr_y_r','gyr_z_r','alpha_r','beta_r']
 
    big_vector_columns=[]
 
    # Loop over the signals
    for signal in signal_list:
 
        new_columns=[]
 
        # Make new column names for each signal
        for column in df.columns.drop(['frame','signal']):
            new_column=str(signal+'_'+column)
            new_columns.append(new_column)
 
        # Add to the column list
        big_vector_columns=big_vector_columns+new_columns
 
    # Create the big vector dataframe
    df_ef_big_vector=pd.DataFrame()
    frame=int(0)
    frame_features={}
 
    # Loop over the frames
    for frame_nr in df['frame'].unique():
        signal_features=[]
 
        # Loop over the signals
        for i in range(0,len(signal_list)):
            #print (frame_nr,(i+frame_nr*14)-14)
            df_row=df.loc[(df['frame'] == frame_nr) & (df['signal'] == signal_list[i])].drop(['frame','signal'],axis=1)
 
            signal_features.append(df_row.values)
 
        # Flatten the signal_features
        for iterations in range(2):
            signal_features=list(itertools.chain.from_iterable(signal_features)) 
        # Append the signal_features to the frame features dictionary
        frame_features[frame_nr]=signal_features
        #print (frame_features[frame_nr])
        #frame_features[frame_nr]=list(itertools.chain.from_iterable(signal_features[frame_nr]))
 
    df_frame_features=pd.DataFrame(frame_features)
    # Concatenate all the features per frame
    for frame_nr in df['frame'].unique():
        df_ef_big_vector=pd.concat([df_ef_big_vector,pd.DataFrame(data=frame_features[frame_nr])],axis=1)
 
    # Transpose and add column names
    df_ef_big_vector=df_ef_big_vector.T
    df_ef_big_vector.columns=big_vector_columns
 
    # Reset the index
    df_ef_big_vector.reset_index(inplace=True,drop=True)
    df_ef_big_vector['frame']=df_ef_big_vector.index+1
    return df_ef_big_vector   

In [6]:
# RUN THE PIPELINE

datapath=os.path.join('../data/unlabeled')
file_nr=int(0)
total_files=len(listdir(datapath))
for file in listdir(datapath):
    if '.csv' in file:
        df=pd.read_csv(os.path.join(datapath,file))
        file_nr+=1 # Counter
    

        # RESAMPLING
        # Create an array of index values with a fixed step
        time_index = np.linspace(0, (len(df)-1)/sample_rate, num=len(df))

        # Use this array as the index
        df.index = time_index

        # Resample the dataframe at (resample_rate) Hz
        df_resampled = df.groupby(np.arange(len(df))//(sample_rate/resample_rate)).mean() 
    
        df=df_resampled 

        # Get the signals from the file
        signal_start= int(125/resample_rate)
        signal_end  = len(df)-500

        acc_x_n,acc_y_n,acc_z_n,gyr_x_n,gyr_y_n,gyr_z_n,acc_x_r,acc_y_r,acc_z_r,gyr_x_r,gyr_y_r,gyr_z_r,alpha_r,beta_r,alpha_min_beta=get_signals(df,signal_start,signal_end,'haar')
        
        # SEGMENTATION
        # Get the changepoints
        results = get_changepoints()

        # Make seperate frames from the change points

        offset=signal_start
        change_points=[]
        start_point=[]
        end_point=[]

        results = get_changepoints()

        for r in results:
            change_points.append(int(r))

        start_point.append(0 + offset)
        for i in range(0,len(change_points)-1,1):
            #print (i)
            end_point.append( change_points[i] + offset + resample_rate)
            start_point.append( change_points[i] + offset - resample_rate)
        end_point.append(change_points[len(change_points)-1] + offset )
        print (f'{len(change_points)} frames detected')

        #PLOT THE GRAPH
        range_start=1000
        range_end=3500
        
        plot_changepoints(acc_x_n,gyr_x_n,acc_z_n,alpha_r,0,alpha_min_beta,results,range_start,range_end,file)

        frame_data=[]

        variables = {
            'acc_x_n': acc_x_n,
            'acc_y_n': acc_y_n,
            'acc_z_n': acc_z_n,
            'gyr_x_n': gyr_x_n,
            'gyr_y_n': gyr_y_n,
            'gyr_z_n': gyr_z_n,
            'acc_x_r': acc_x_r,
            'acc_y_r': acc_y_r,
            'acc_z_r': acc_z_r,
            'gyr_x_r': gyr_x_r,
            'gyr_y_r': gyr_y_r,
            'gyr_z_r': gyr_z_r,
            'alpha_r': alpha_r,
            'beta_r': beta_r,
            'alpha_min_beta': alpha_min_beta
        }

        complete_df=pd.DataFrame()

        for i in range(len(change_points)): 
            
            signals_frame = get_signals_range(variables, start_point[i],end_point[i])
            acc_x_n_frame = signals_frame['acc_x_n']
            acc_y_n_frame = signals_frame['acc_y_n']
            acc_z_n_frame = signals_frame['acc_z_n']
            gyr_x_n_frame = signals_frame['gyr_x_n']
            gyr_y_n_frame = signals_frame['gyr_y_n']
            gyr_z_n_frame = signals_frame['gyr_z_n']
            acc_x_r_frame = signals_frame['acc_x_r']
            acc_y_r_frame = signals_frame['acc_y_r']
            acc_z_r_frame = signals_frame['acc_z_r']
            gyr_x_r_frame = signals_frame['gyr_x_r']
            gyr_y_r_frame = signals_frame['gyr_y_r']
            gyr_z_r_frame = signals_frame['gyr_z_r']
            alpha_r_frame = signals_frame['alpha_r']
            beta_r_frame = signals_frame['beta_r']
            alpha_min_beta_frame = signals_frame['alpha_min_beta']

            frame_data.append(np.vstack([acc_x_n_frame,acc_y_n_frame,acc_z_n_frame,acc_x_r_frame,acc_y_r_frame,acc_z_r_frame,gyr_x_n_frame,gyr_y_n_frame,gyr_z_n_frame,gyr_x_r_frame,gyr_y_r_frame,gyr_z_r_frame,alpha_r_frame,beta_r_frame]).T)
            df1=pd.DataFrame(data=frame_data[i], columns=['acc_x_n','acc_y_n','acc_z_n','acc_x_r','acc_y_r','acc_z_r','gyr_x_n','gyr_y_n','gyr_z_n','gyr_x_r','gyr_y_r','gyr_z_r','alpha_r','beta_r'])
            df1['frame']=i
            df1['ts']=df1.index

            complete_df=pd.concat([complete_df,df1],axis=0)
            #if i%15==0:
                #print(start_point[i],end_point[i])
                #plot_frame(acc_x_n_frame,acc_y_n_frame,acc_z_n_frame,gyr_x_n_frame,gyr_y_n_frame,gyr_z_n_frame,alpha_r_frame,beta_r_frame,i)
                
        complete_df.to_csv(f'../data/processed/unlabeled/frames/frames_{file}')

        # FEATURE EXTRACTION
        # Extract features with tsfresh
        extracted_features={}
        signals=['acc_x_n','acc_y_n','acc_z_n','gyr_x_n','gyr_y_n','gyr_z_n','alpha_r','beta_r'] 
        df_ef=pd.DataFrame()

        for i in range(len(frame_data)):

            df_fd=pd.DataFrame(data=frame_data[i], columns=['acc_x_n','acc_y_n','acc_z_n','acc_x_r','acc_y_r','acc_z_r','gyr_x_n','gyr_y_n','gyr_z_n','gyr_x_r','gyr_y_r','gyr_z_r','alpha_r','beta_r'])
            df_fd['frame']=i
            df_fd['ts']=df_fd.index

            signals=columns=['acc_x_n','acc_y_n','acc_z_n','acc_x_r','acc_y_r','acc_z_r','gyr_x_n','gyr_y_n','gyr_z_n','gyr_x_r','gyr_y_r','gyr_z_r','alpha_r','beta_r']
            extracted_features={}   
                
            cnt=int(0)
            for signal in signals:
                if __name__ == "__main__":
                    extracted_features[signal] = extract_features(df_fd, 
                                        column_id="frame", column_sort="ts", column_value=signal, 
                                        impute_function=impute, show_warnings=False, 
                                        n_jobs=0)

                    extracted_features[signal]['frame']=i+1
                    extracted_features[signal]['signal']=signal
                    df_ef_signal=pd.DataFrame(data=extracted_features[signal].values)
                    df_ef=pd.concat([df_ef,df_ef_signal],axis=0)

        # Make clean column names for df_ef 
        clean_columns=[]
        columns=extracted_features['acc_x_n'].columns
        for column in columns:
            clean_column=re.sub('acc_x_n__','',column)
            clean_columns.append(clean_column)
        df_ef.columns=clean_columns
        df_ef.reset_index(drop=True, inplace=True)
        
        # Combine all the extracte features per frame into a 1D vector
        df_ef_big_vector=get_extracted_features_table(df_ef)
        df_ef_big_vector

        # Write these big vectors to a file
        df_ef_big_vector.to_csv(f'../data/processed/unlabeled/ts/ts_feat_{file}')

        print (f'[{file_nr}/{total_files}] Sucessfully processed file {file}')


102 frames detected


Feature Extraction: 100%|██████████| 1/1 [00:01<00:00,  1.07s/it]


[1/128] Sucessfully processed file 059-COMBINED-2022-11-7-13-2-45-1667849080-v1.4.3+lucina-buzz.csv
84 frames detected


Feature Extraction: 100%|██████████| 1/1 [00:00<00:00,  8.33it/s]


[2/128] Sucessfully processed file 059-COMBINED-2022-11-7-15-0-54-1667850876-v1.4.3+lucina-buzz.csv
80 frames detected


...

[15/128] Sucessfully processed file 065-COMBINED-2022-10-26-9-29-23-1666808223-v1.4.3+lucina-buzz.csv
87 frames detected


Feature Extraction: 100%|██████████| 1/1 [00:01<00:00,  1.80s/it]


[16/128] Sucessfully processed file 065-COMBINED-2022-10-27-6-44-47-1666873618-v1.4.3+lucina-buzz.csv
99 frames detected


Feature Extraction: 100%|██████████| 1/1 [00:00<00:00, 11.14it/s]


[17/128] Sucessfully processed file 065-COMBINED-2022-10-27-9-18-5-1666873810-v1.4.3+lucina-buzz.csv
